# 函数准备

In [1]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import random
from random import choice
random.seed(10)
# -*- coding:utf-8 -*
import eval
from mkdir import mk_dir

# 训练集准备

In [1]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10)

In [8]:
# for col_ab in ["age", "gender","num","evi","immu"]:
for col_ab in ['pat']:

    mk_dir("eval/evl_%s" % (col_ab))

    train_set = json.load(open("input/input_%s/train_set.json" \
        %(col_ab), "r"))
    train_qa = train_set['data'][0]["paragraphs"]

    re = 0
    for ttrain_qa,eval_qa in kf.split(train_qa):

        re = re + 1

        ttrain_set = {
            "data": [{
                'paragraphs': [train_qa[i] for i in ttrain_qa],
                'title': "LungB"
            }],
            "version": "LungB"
        }

        mk_dir("eval/evl_%s/input_%s" % (col_ab, re))
        mk_dir("eval/evl_%s/output_%s" % (col_ab, re))

        json.dump(ttrain_set, open("eval/evl_%s/input_%s/train_set.json" 
        % (col_ab, re), "w"))

        eval_q=[]
        eval_a={}
        for i in eval_qa:
            qq={
                'qas': [{
                    'id': train_qa[i]['qas'][0]['id'],
                    'question': train_qa[i]['qas'][0]['question'],
                    'answers':[]
                }],
                'context': train_qa[i]['context']
            }
            eval_q.append(qq)
            eval_a[train_qa[i]['qas'][0]['id']] = train_qa[i]['qas'][0]['answers']

        eval_qset = {
            "data": [{
                'paragraphs': eval_q,
                'title': "LungB"
            }],
            "version": "LungB"
        }

        json.dump(eval_qset, open("eval/evl_%s/input_%s/test_set.json" 
        % (col_ab, re), "w"))
        json.dump(eval_a, open("eval/evl_%s/input_%s/eval_set.json" 
        % (col_ab, re), "w"))

In [ ]:
col_ab =  "pat"
standard_0 = pd.DataFrame(np.zeros((0,1)), columns = ["golden"])
prediction_0 = pd.DataFrame(np.zeros((0,3)), columns = ["prediction","rate","cut"])
for re in range(1,11):
    eval_a = json.load(open("eval/evl_%s/input_%s/eval_set.json" % (col_ab, re), "r"))
    standard = pd.DataFrame(eval_a, index=["golden"]).T
    standard_0 = pd.concat((standard_0, standard), 0)
    prediction = json.load(open("eval/evl_%s/output_%s/predictions_.json" % (col_ab, re),'r'))
    prediction = pd.DataFrame(prediction, index=["prediction", "rate"]).T
    prediction["cut"] = 1
    for idx in prediction.index:
        prediction.loc[idx, "rate"] = prediction.loc[idx, "rate"][0]
        prediction.loc[idx, "cut"] = 1 - prediction.loc[idx, "rate"]
        if prediction.loc[idx, "rate"] < 0.85:
            prediction.loc[idx, "prediction"] = "no"
    prediction_0 = pd.concat((prediction_0, prediction),0)

fig_path = ("eval/evl_%s/output_%s" % (col_ab, re))
eval.evaluate(standard_0, prediction_0, fig_path)